In [2]:
import pandas as pd
from pandas import DataFrame, read_csv
import networkx as nx

In [87]:


df = pd.read_csv('vb_1_edges_foro_completo.csv',sep=';')
df.head()

,Source,Target,topic_id,weight,relationship
0,glaros,Benjamin Stevens,1,2,replies_to_author
1,RunnerPack,thunderstruck,0,22,replies_to_author
2,Seam,DogP,2,1,replies_to_author
3,RetroDan,KGRAMR,2,2,replies_to_author
4,ZueriHB,Atari26003,6,1,replies_to_author


In [88]:
df['topic_id'].unique()

array([ 1,  0,  2,  6,  5,  4,  3,  7, 11,  9], dtype=int64)

In [89]:
df.loc[df['topic_id'] == 2, 'topic_id'] = 'Social'

In [90]:
df.head()

,Source,Target,topic_id,weight,relationship
0,glaros,Benjamin Stevens,1,2,replies_to_author
1,RunnerPack,thunderstruck,0,22,replies_to_author
2,Seam,DogP,Social,1,replies_to_author
3,RetroDan,KGRAMR,Social,2,replies_to_author
4,ZueriHB,Atari26003,6,1,replies_to_author


In [91]:
df.loc[df['topic_id'] == 5, 'topic_id'] = 'Marketplace'
df.loc[df['topic_id'] == 0, 'topic_id'] = 'Dev'
df.loc[df['topic_id'] == 4, 'topic_id'] = 'Hardware/Repairs'
df.loc[df['topic_id'] == 7, 'topic_id'] = 'Non-English'

df.loc[df['topic_id'] == 6, 'topic_id'] = 'Misc'
df.loc[df['topic_id'] == 8, 'topic_id'] = 'Misc'
df.loc[df['topic_id'] == 9, 'topic_id'] = 'Misc'
df.loc[df['topic_id'] == 10, 'topic_id'] = 'Misc'

df.loc[df['topic_id'] == 1, 'topic_id'] = 'Information/Scans'
df.loc[df['topic_id'] == 3, 'topic_id'] = 'Information/Scans'

df.loc[df['topic_id'] == 11, 'topic_id'] = 'Information/Scans'

In [92]:
df['topic_id'].unique()

array(['Information/Scans', 'Dev', 'Social', 'Misc', 'Marketplace',
       'Hardware/Repairs', 'Non-English'], dtype=object)

In [93]:
df = df[df['relationship'] == 'replies_to_author']

In [94]:
df.groupby(['topic_id']).agg({'weight': 'sum'}).apply(lambda x:
                                                 100 * x / float(x.sum()))

,weight
topic_id,
Dev,21.016130
Hardware/Repairs,11.437403
Information/Scans,2.817739
Marketplace,27.218325
Misc,5.183688
Non-English,2.215353
Social,30.111362


In [79]:
import networkx as nx

In [80]:
G = nx.Graph()
for index, row in df.iterrows():
    
    G.add_node(row['Source'], bipartite=0)
    G.add_node(row['topic_id'], bipartite=1)
    
    if G.has_edge(row['Source'], row['topic_id']):
                        G[row['Source']][row['topic_id']]['weight'] += row['weight']
    else:
        G.add_edge(row['Source'], row['topic_id'], weight = row['weight'])
    

In [81]:
nx.is_bipartite(G)

True

In [82]:
len(list(G.edges()))

2542

In [83]:
nx.write_graphml_lxml(G, "user_topics_VB.graphml")

In [79]:
G = nx.read_graphml("user_topics_VB.graphml")

In [80]:
top_nodes = {n for n, d in G.nodes(data=True) if d['bipartite']==1}
bottom_nodes = set(G) - top_nodes

In [85]:
from networkx.algorithms import bipartite

G_top = bipartite.collaboration_weighted_projected_graph(G, top_nodes)
G_bottom = bipartite.collaboration_weighted_projected_graph(G, bottom_nodes)

In [86]:
len(list(G_bottom.nodes()))

1162

In [87]:
nx.write_graphml_lxml(G_top, "user_topics_VB_top.graphml")
nx.write_graphml_lxml(G_bottom, "user_topics_VB_bottom.graphml")

In [83]:
import community
partition = community.best_partition(G_bottom)

In [84]:
size = float(len(set(partition.values())))

In [85]:
size

5.0

In [95]:
df['community'] = df.apply(lambda row: partition[row.Source], axis=1)

In [96]:
df.head()

,Source,Target,topic_id,weight,relationship,community
0,glaros,Benjamin Stevens,Information/Scans,2,replies_to_author,3
1,RunnerPack,thunderstruck,Dev,22,replies_to_author,0
2,Seam,DogP,Social,1,replies_to_author,0
3,RetroDan,KGRAMR,Social,2,replies_to_author,3
4,ZueriHB,Atari26003,Misc,1,replies_to_author,0


In [103]:
df.to_csv('communities_VB_tab.csv', sep ='\t')

In [97]:
df.groupby(['community']).agg({'weight': 'sum'}).apply(lambda x:
                                                 100 * x / float(x.sum()))

,weight
community,
0,33.690009
1,3.404272
2,34.149725
3,17.401815
4,11.354179


In [98]:
pesos = df.groupby(['community', 'topic_id']).agg({'weight': 'sum'})

In [99]:
pesos['percentage'] = pesos['weight']/pesos.groupby('community')['weight'].transform('sum')

In [100]:
pesos

weight  percentage
community topic_id                             
0         Dev                  3419    0.402188
          Hardware/Repairs      895    0.105282
          Information/Scans     245    0.028820
          Marketplace          1185    0.139395
          Misc                  423    0.049759
          Non-English           110    0.012940
          Social               2224    0.261616
1         Dev                    49    0.057043
          Hardware/Repairs       43    0.050058
          Information/Scans       4    0.004657
          Marketplace           131    0.152503
          Misc                   57    0.066356
          Non-English           341    0.396973
          Social                234    0.272410
2         Dev                   848    0.098410
          Hardware/Repairs      518    0.060114
          Information/Scans     274    0.031798
          Marketplace          4162    0.482999
          Misc                  496    0.057561
          Non-English            68    0.007891
          Social               2251    0.261228
3         Dev                   488    0.111136
          Hardware/Repairs      510    0.116147
          Information/Scans     123    0.028012
          Marketplace           886    0.201776
          Misc                  205    0.046686
          Non-English            27    0.006149
          Social               2152    0.490093
4         Dev                   499    0.174171
          Hardware/Repairs      920    0.321117
          Information/Scans      65    0.022688
          Marketplace           504    0.175916
          Misc                  127    0.044328
          Non-English            13    0.004538
          Social                737    0.257243

In [97]:
df_gephi = pd.read_csv('modularity_gephi.csv')
df_gephi.head()

,Id,Label,timeset,d0,modularity_class
0,Rocksteady,Rocksteady,NaN,0,5
1,vb zach,vb zach,NaN,0,3
2,M.K.,M.K.,NaN,0,0
3,Marc,Marc,NaN,0,3
4,trippstatic,trippstatic,NaN,0,0


In [98]:
dict_gephi = dict(zip(df_gephi['Id'], df_gephi['modularity_class']))

In [99]:
dict_gephi

{'Rocksteady': 5,
 'vb zach': 3,
 'M.K.': 0,
 'Marc': 3,
 'trippstatic': 0,
 'El Borto': 3,
 'Alfissta': 0,
 'boom': 1,
 'introvert': 5,
 'ryucheese': 1,
 'rdk123': 1,
 'Nessie': 0,
 'Hiro': 1,
 'Virtual Jim': 5,
 'Dr. Jeckidy': 2,
 'D.R MARIO': 5,
 'zeromous': 2,
 'jaku': 3,
 'decayedmatter': 0,
 'dragz': 2,
 'Florian': 5,
 'GabrielHanson2000': 0,
 'Tokimemofan': 2,
 'ThatOtherDan': 2,
 'Pierce': 4,
 'Raijin Z': 1,
 'alesi80': 4,
 'Rogueleader1101': 2,
 'InactiveX': 0,
 'Vitual Jim': 2,
 'Pre10der': 1,
 'Sega_gw': 3,
 'Virtual_Link': 0,
 'David': 3,
 'Xinorok': 3,
 'Nitrosoxide': 4,
 'oldskoolmodderguy': 1,
 'Ferris Bueller': 3,
 'Hungry Mech': 5,
 'CZroe': 3,
 'shenmueso': 0,
 'Samuel': 4,
 'MAXLEFOU': 5,
 'coleco': 2,
 'PsychedelicShaman': 2,
 'megax9': 2,
 'Wario': 3,
 'JKouf23': 1,
 'Nightowljrm': 0,
 'DocDoak': 5,
 'TheCamel': 2,
 'cualquiercosa327': 4,
 'Supa~Squrriel': 5,
 'jmo': 4,
 '8bitmatt': 0,
 'Russcular': 3,
 'CyberAkuma': 1,
 'Dead Head': 4,
 'thunderstruck': 0,
 'Magnu

In [100]:
df['community_gephi'] = df.apply(lambda row: dict_gephi[row.Source], axis=1)

In [101]:
df.head()

,Source,Target,topic_id,weight,relationship,community,community_gephi
0,glaros,Benjamin Stevens,Information/Scans,2,replies_to_author,2,0
1,RunnerPack,thunderstruck,Dev,22,replies_to_author,2,0
2,Seam,DogP,Social,1,replies_to_author,2,0
3,RetroDan,KGRAMR,Social,2,replies_to_author,2,0
4,ZueriHB,Atari26003,Misc,1,replies_to_author,2,0


In [102]:
pesos_gephi = df.groupby(['community_gephi', 'topic_id']).agg({'weight': 'sum'})
pesos_gephi['percentage'] = pesos_gephi['weight']/pesos_gephi.groupby('community_gephi')['weight'].transform('sum')
pesos_gephi

weight  percentage
community_gephi topic_id                             
0               Dev                  4375    0.226966
                Hardware/Repairs     2211    0.114702
                Information/Scans     709    0.036781
                Marketplace          5199    0.269714
                Misc                  983    0.050996
                Non-English           194    0.010064
                Social               5605    0.290776
1               Hardware/Repairs      264    0.423756
                Marketplace           165    0.264848
                Misc                   32    0.051364
                Social                162    0.260032
2               Misc                   28    0.074271
                Social                349    0.925729
3               Marketplace           574    0.648588
                Misc                   50    0.056497
                Social                261    0.294915
4               Dev                   795    0.284335
                Hardware/Repairs      332    0.118741
                Marketplace           647    0.231402
                Misc                  135    0.048283
                Social                887    0.317239
5               Dev                   133    0.104232
                Hardware/Repairs       79    0.061912
                Information/Scans       2    0.001567
                Marketplace           283    0.221787
                Misc                   80    0.062696
                Non-English           365    0.286050
                Social                334    0.261755

# NEWMAN CON PESOS

In [8]:
def my_weight(G, u, v, weight='weight'):
    w = 0
    for nbr in set(G[u]) & set(G[v]):
        w += G[u][nbr].get(weight, 1) + G[v][nbr].get(weight, 1)
    return w

In [9]:
edges = [('A1','B1',3),
         ('A1','B2',7),
         ('A2','B1',2),
         ('A2','B2',4),
          ('A3','B1',3),
          ('A3','B3',2),
         ('A1','B3',2)
         ]

B = nx.Graph()
B.add_weighted_edges_from(edges)

In [12]:
from networkx.algorithms import bipartite
G = bipartite.generic_weighted_projected_graph(B, ['A1', 'A2'], weight_function=my_weight)


print(list(G.edges(data=True)))

[('A1', 'A2', {'weight': 16}), ('A1', 'A3', {'weight': 10}), ('A2', 'A3', {'weight': 5})]


In [77]:
def forum_projection(B, nodes):
    if B.is_directed():
        pred = B.pred
        G = nx.DiGraph()
    else:
        pred = B.adj
        G = nx.Graph()
    G.graph.update(B.graph)
    G.add_nodes_from((n, B.nodes[n]) for n in nodes)
    for u in nodes:
        unbrs = set(B[u]) #nodos a los que conecta del otro nivel
        nbrs2 = set(n for nbr in unbrs for n in B[nbr] if n != u) #nodos con los que coincide
        for v in nbrs2:
            vnbrs = set(pred[v])
            common_degree = ((len(B[n]), B[u][n]['weight'] + B[v][n]['weight']) for n in unbrs & vnbrs)
            weight = sum(wt / (deg - 1) for deg, wt in common_degree if deg > 1)
            G.add_edge(u, v, weight=weight)
    return G

In [78]:
G = forum_projection(B, ['A1', 'A2'])


print(list(G.edges(data=True)))

[('A1', 'A2', {'weight': 13.5}), ('A1', 'A3', {'weight': 7.0}), ('A2', 'A3', {'weight': 2.5})]


In [19]:
unbrs = set(B['A1'])

In [21]:
set(n for nbr in unbrs for n in B[nbr] if n != 'A1')

{'A2', 'A3'}

In [24]:
pred = B.adj

vnbrs = set(pred['A2'])

In [73]:
common_degree = ((len(B[n]), B['A1'][n]['weight']) for n in unbrs & vnbrs)

In [69]:
common_degree = (B['A1'][n] for n in unbrs & vnbrs)

In [74]:
list(common_degree)

[(2, 7), (3, 3)]

In [58]:
sum((deg - 1) for deg in common_degree if deg > 1)

3

In [59]:
unbrs

{'B1', 'B2', 'B3'}

In [60]:
vnbrs

{'B1', 'B2'}

In [64]:
B['A1']['B1']['weight']

3

In [81]:
G_top = forum_projection(G, top_nodes)
G_bottom = forum_projection(G, bottom_nodes)

In [82]:
nx.write_graphml_lxml(G_top, "user_topics_VB_top_newman.graphml")
nx.write_graphml_lxml(G_bottom, "user_topics_VB_bottom_newman.graphml")